### This is just the basic model without feats for the sensitivity prediction.

Again, please install the needed resources 

This code is basically identical to the first sensitivity model. I will use documentation more sparingly.

There might be some incompatibilities, because I had to switch from SageMaker to Google Colab. If anything doesn't run, it might just be a forgotten import and in the 
worst case a forgotten variable, but I don't want to destroy the outputs and rerun the cells. A copy of this worked for me, but it was in the same runtime environment.

In [2]:
import tensorflow as tf
import torch
import numpy as np
from transformers import BertTokenizer
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import pandas as pd
import random
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
##device = torch.device('cpu')
print('We will use the This:', torch.cuda.get_device_name(0))

We will use the This: Tesla T4


In [43]:
df = pd.read_csv("experiment.tsv", sep='\t', header=None, names=['Text', 'Privacy'])
df = pd.DataFrame(df)
df.head()

,Text,Privacy
0,Drug challenge idea: take shrooms or some LSD ...,0
1,Austin pulliam making rude gestures toward me....,0
2,"Got my mind thinking, pretty girls drinking ta...",0
3,"""@XXX: I think white and have nigger lips"" mad...",0
4,You need to swallow some make-up or something ...,0


In [44]:
# Train and Test splits
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.Privacy.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.Privacy.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Privacy', 'data_type']).count()

Text
Privacy data_type      
0       train      1219
        val         216
1       train      1220
        val         215

In [45]:
# Tokenization process
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values, 
    add_special_tokens=True, 
    truncation=True,
    return_attention_mask=True,  
    max_length=80, 
    padding='max_length',
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values, 
    add_special_tokens=True, 
    truncation=True,
    return_attention_mask=True, 
    max_length=80, 
    padding='max_length',
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Privacy.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Privacy.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [46]:
# Still binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [47]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [48]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [27]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [50]:
# IMORTANT
# This time the folder is called "sensitivity_nofeats"

seed_val = 47
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Evaluation and training again
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'sensitivity_nofeats/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/77 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.502113009428049
Validation loss: 0.38132152067763464
F1 Score (Weighted): 0.8272631157226247


Epoch 2:   0%|          | 0/77 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.38449663039925813
Validation loss: 0.30260458695037024
F1 Score (Weighted): 0.8722646325703847


...


Epoch 3
Training loss: 0.2967073693871498
Validation loss: 0.2921949880463736
F1 Score (Weighted): 0.8698299281418242


Epoch 4:   0%|          | 0/77 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.25479736643565165
Validation loss: 0.2777229765696185
F1 Score (Weighted): 0.8607738728931627


Epoch 5:   0%|          | 0/77 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.22131708477224624
Validation loss: 0.288465074130467
F1 Score (Weighted): 0.874478859279616


In [51]:
# Load the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('sensitivity_nofeats/finetuned_BERT_epoch_4.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [52]:
# Print the accuracies
pr_right, pr_wrong, pu_right, pu_wrong = 0, 0, 0, 0 
for i, pred in enumerate(predictions):
  if(true_vals[i]==0):
    if(np.argmax(pred) == true_vals[i]):
      pr_right += 1
    else:
      pr_wrong +=1
  else:
    if(np.argmax(pred) == true_vals[i]):
      pu_right += 1
    else:
      pu_wrong +=1

pr_acc = round(100*(pr_right/(pr_right+pr_wrong)), 2)
pu_acc = round(100*(pu_right/(pu_right+pu_wrong)), 2)
print("Private Tweets Accuracy:")
print("{}/{} {}%".format(pr_right,pr_wrong+pr_right,pr_acc))
print("Public Tweets Accuracy:")
print("{}/{} {}%".format(pu_right,pu_wrong+pu_right,pu_acc))

Private Tweets Accuracy:
188/216 87.04%
Public Tweets Accuracy:
183/215 85.12%


In [53]:
# Show the confusion matrix
pred_np = np.zeros(len(predictions))
for i, pred in enumerate(predictions):
  pred_np[i] = np.argmax(pred)

In [54]:
conf_mx = metrics.confusion_matrix(true_vals, pred_np)
conf_mx

array([[188,  28],
       [ 32, 183]])